# <font color='#114B5F'><p style="text-align: center;">Smart meters in London</p></font>
<font color='#114B5F'>NOTE:</font> For visualizations, please refer to the "Smart_meter_london_visualizations.ipynb".

## <font color='#114B5F'><p><b>Table of Contents</b></p></font>
<ol>
    <li><a href='#overview-anchor'>Overview</a></li>
    <li><a href='#library-anchor'>Import libraries</a></li>
    <li><a href='#weather-anchor'>Reading & cleaning weather dataset</a>
        <ol style="list-style-type: lower-roman;"><li>creating datetime
            <li>creating weather dataframe for 2013
            <li> write to csv
        </ol>
    </li>
    <li><a href='#household-anchor'>Reading Household dataset</a>
    <li><a href='#block-anchor'>Reading & cleaning Block dataset</a>
        <ol style="list-style-type: lower-roman;"><li>Renaming Columns
            <li>creating datetime
            <li>Merging with household dataframe
                <li>write to csv
                    
        </ol>
    </li>
</ol>

## <font color='#114B5F'><p style="text-align: center;"><a id='overview-anchor'></a>Overview</p></font>
<font color='#114B5F'><p>The goal of this analysis is to understand energy consumption pattern in UK and factors influencing it.This dataset was used to understand energy consumption patterns across different ACORN groups. Initial analysis consisted of understanding the different ACORN groups and the difference between them. The next steps of analysis consisted of a deep dive into the dataset which contains the records of household energy consumption (monitored every half hour) to try to establish different consumption patterns based on tariff plans and the ACORN groups. Analysis of weather impacts on energy consumption across groups was also conducted. The resulting output comprised of visualizations of various parameters which exhibit energy consumption trends within and across ACORN groups.</p>

## <font color='#114B5F'><p style="text-align: center;"><a id='library-anchor'></a>Import libraries</p></font>

In [1]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import pandas as pd
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings("ignore")

## <font color='#114B5F'><p style="text-align: center;"><a id='weather-anchor'></a>Reading & Cleaning Weather Dataset</p></font>

<font color='#114B5F'><b> Weather_hourly_darksky.csv :</b> </font>
Contains hourly data from darksky api. It consists of hourly weather information across UK including minimum-maximum temperature and wind data.

In [3]:
# Reading Weather data
df_weather = pd.read_csv("FinalData_SmartMeter/weather_hourly_darksky.csv")

In [4]:
# Creating date, time related columns
df_weather = df_weather[['temperature', 'time']]
df_weather.columns = ['temperature', 'DateTime']
df_weather['DateTime'] = pd.to_datetime(df_weather['DateTime'])
df_weather['year'] = df_weather['DateTime'].dt.year
df_weather['month'] = df_weather['DateTime'].dt.month
df_weather['day'] = df_weather['DateTime'].dt.day

In [5]:
#### Weather information for the year 2013
df_weather_2013 = df_weather[df_weather.year==2013][['temperature', 'month']]
df_weather_2013 = df_weather_2013.groupby(by=['month'])['temperature'].mean().to_frame()
df_weather_2013.reset_index(level=0, inplace=True)
df_weather_2013.month = df_weather_2013.month.apply(lambda x: calendar.month_abbr[x])

In [6]:
# Creating a CSV file.
df_weather_2013.to_csv("weather.csv", index=False, sep=',', encoding='utf-8')

## <font color='#114B5F'><p style="text-align: center;"><a id='household-anchor'></a>Reading Household dataset</p></font>

<font color='#114B5F'><b> Informations_households.xls :</b> </font>
Contains all the information on the households including type of acorn group and type of tariff plan used(Standard/ Dynamic Time of Use) in each household respectively.

In [2]:
# Reading Household info data
df_household = pd.read_csv("FinalData_SmartMeter/informations_households.xls", encoding="utf-8")

## <font color='#114B5F'><p style="text-align: center;"><a id='block-anchor'></a>Reading & Cleaning Block dataset</p></font>

<font color='#114B5F'><b>Halfhourly_dataset.zip :</b> </font>
<p>Zip file contains the block files with the half-hourly smart meter measurement.  For this analysis, the following blocks were used.</p>
<p>Blocks used : Block_0, Block_2, Block_4, Block_62, Block_78, Block_79, Block_80, Block_95, Block_96, Block_105</p>
<p>For this analysis, households belonging to Affluent Achievers, Comfortable Communities and Urban Adversity ACORN groups were only considered.</p> 

<font color='#114B5F'><b>Block_0 dataset</b> </font>

In [7]:
# Reading Block_0
df_block_0 = pd.read_csv("FinalData_SmartMeter/block_0.csv", engine='python')

In [8]:
# Renaming the columns
df_block_0.columns = ['LCLid', 'DateTime', 'KWh']

In [9]:
# Creating date, time related columns
df_block_0['DateTime'] = pd.to_datetime(df_block_0['DateTime'])
df_block_0['year'] = df_block_0['DateTime'].dt.year
df_block_0['month'] = df_block_0['DateTime'].dt.month
df_block_0['day'] = df_block_0['DateTime'].dt.day
df_block_0['hour'] = df_block_0['DateTime'].dt.hour
df_block_0['date'] = df_block_0['DateTime'].dt.date

In [10]:
# Creating time column which has time in 12 hour with AM/PM format
format = '%I:%M %p'
df_block_0['time'] = df_block_0['DateTime'].dt.time.apply(lambda x: x.strftime(format))

In [11]:
# Cleaning 
df_block_0.loc[(df_block_0.time=='12:32 PM'), 'time'] = '12:30 PM'
df_block_0.loc[(df_block_0.time=='12:37 PM'), 'time'] = '12:30 PM'
df_block_0.loc[(df_block_0.time=='03:13 PM'), 'time'] = '03:00 PM'
df_block_0.loc[(df_block_0.time=='03:15 PM'), 'time'] = '03:00 PM'

<font color='#114B5F'><b>Merge Block 0 with Household_info </b> </font>

In [12]:
# Total no. of households in the datasets
df_household.LCLid.nunique()

5566

In [13]:
# Merging
df_block_0_info = df_block_0.merge(df_household, on='LCLid', how='left')

In [14]:
df_block_0_info.drop(['file'], axis=1, inplace=True)

In [15]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_0_info['Acorn'] = df_block_0_info['Acorn'].str.replace('ACORN-', "")

In [16]:
df_block_0_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000002,2012-10-12 00:30:00,0,2012,10,12,0,2012-10-12,12:30 AM,Std,A,Affluent
1,MAC000002,2012-10-12 01:00:00,0,2012,10,12,1,2012-10-12,01:00 AM,Std,A,Affluent


In [17]:
df_block_0_info['stdorToU'].unique()

array(['Std'], dtype=object)

In [20]:
df_block_0_info['Acorn_grouped'].unique()

array(['Affluent', 'Adversity', 'Comfortable'], dtype=object)

In [21]:
# cleaning(Removing the rows which are classified as 'ACORN-U')
df_block_0_info = df_block_0_info[(df_block_0_info['Acorn_grouped']!='ACORN-U')]


Block_0_info does not have any households that belong to DToU tariff plan. So, to include households 
belonging to DToU tariff plan, following blocks were taken.

<font color='#114B5F'><b>Block 2 dataset</b> </font>

In [22]:
# Reading block_2 data
df_block_2 = pd.read_csv("FinalData_SmartMeter/block_2.csv", engine='python')

In [23]:
# Renaming the columns
df_block_2.columns = ['LCLid', 'DateTime', 'KWh']

In [24]:
# Creating date, time related columns
df_block_2['DateTime'] = pd.to_datetime(df_block_2['DateTime'])
df_block_2['year'] = df_block_2['DateTime'].dt.year
df_block_2['month'] = df_block_2['DateTime'].dt.month
df_block_2['day'] = df_block_2['DateTime'].dt.day
df_block_2['hour'] = df_block_2['DateTime'].dt.hour
df_block_2['date'] = df_block_2['DateTime'].dt.date

In [25]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_2['time'] = df_block_2['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 2 with Household_info </b> </font>

In [26]:
df_block_2_info = df_block_2.merge(df_household, on='LCLid', how='left')

In [27]:
df_block_2_info.drop(['file'], axis=1, inplace=True)

In [28]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_2_info['Acorn'] = df_block_2_info['Acorn'].str.replace('ACORN-', "")

In [29]:
df_block_2_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000030,2011-12-07 11:30:00,0.398,2011,12,7,11,2011-12-07,11:30 AM,Std,A,Affluent
1,MAC000030,2011-12-07 12:00:00,0.578,2011,12,7,12,2011-12-07,12:00 PM,Std,A,Affluent


In [30]:
df_block_2_info['Acorn_grouped'].unique()

array(['Affluent'], dtype=object)

<font color='#114B5F'><b>Block 4 dataset </b> </font>

In [31]:
# Reading block_4 data
df_block_4 = pd.read_csv("FinalData_SmartMeter/block_4.csv", engine='python')

In [32]:
# Renaming the columns
df_block_4.columns = ['LCLid', 'DateTime', 'KWh']

In [33]:
# Creating date, time related columns
df_block_4['DateTime'] = pd.to_datetime(df_block_4['DateTime'])
df_block_4['year'] = df_block_4['DateTime'].dt.year
df_block_4['month'] = df_block_4['DateTime'].dt.month
df_block_4['day'] = df_block_4['DateTime'].dt.day
df_block_4['hour'] = df_block_4['DateTime'].dt.hour
df_block_4['date'] = df_block_4['DateTime'].dt.date

In [34]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_4['time'] = df_block_4['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 4 with Household_info </b> </font>

In [35]:
df_block_4_info = df_block_4.merge(df_household, on='LCLid', how='left')

In [36]:
df_block_4_info.drop(['file'], axis=1, inplace=True)

In [37]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_4_info['Acorn'] = df_block_4_info['Acorn'].str.replace('ACORN-',"")

In [38]:
df_block_4_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000517,2012-03-27 11:30:00,0.305,2012,3,27,11,2012-03-27,11:30 AM,Std,C,Affluent
1,MAC000517,2012-03-27 12:00:00,0.118,2012,3,27,12,2012-03-27,12:00 PM,Std,C,Affluent


In [39]:
df_block_4_info['Acorn_grouped'].unique()

array(['Affluent'], dtype=object)

<font color='#114B5F'><b>Block 62 dataset</b> </font>

In [40]:
# Reading block_62 data
df_block_62 = pd.read_csv("FinalData_SmartMeter/block_62.csv", engine='python')

In [41]:
# Renaming columns
df_block_62.columns = ['LCLid', 'DateTime', 'KWh']

In [42]:
# Creating date, time related columns
df_block_62['DateTime'] = pd.to_datetime(df_block_62['DateTime'])
df_block_62['year'] = df_block_62['DateTime'].dt.year
df_block_62['month'] = df_block_62['DateTime'].dt.month
df_block_62['day'] = df_block_62['DateTime'].dt.day
df_block_62['hour'] = df_block_62['DateTime'].dt.hour
df_block_62['date'] = df_block_62['DateTime'].dt.date

In [43]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_62['time'] = df_block_62['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 62 with Household_info </b> </font>

In [44]:
df_block_62_info = df_block_62.merge(df_household, on='LCLid', how='left')

In [45]:
df_block_62_info.drop(['file'], axis=1, inplace=True)

In [46]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_62_info['Acorn'] = df_block_62_info['Acorn'].str.replace('ACORN-', "")

In [47]:
df_block_62_info.head()


,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000010,2012-09-24 12:30:00,0.918,2012,9,24,12,2012-09-24,12:30 PM,Std,H,Comfortable
1,MAC000010,2012-09-24 13:00:00,0.973,2012,9,24,13,2012-09-24,01:00 PM,Std,H,Comfortable
2,MAC000010,2012-09-24 13:30:00,0.515,2012,9,24,13,2012-09-24,01:30 PM,Std,H,Comfortable
3,MAC000010,2012-09-24 14:00:00,0.281,2012,9,24,14,2012-09-24,02:00 PM,Std,H,Comfortable
4,MAC000010,2012-09-24 14:30:00,0.28,2012,9,24,14,2012-09-24,02:30 PM,Std,H,Comfortable


In [48]:
df_block_62_info['Acorn_grouped'].unique()

array(['Comfortable'], dtype=object)

<font color='#114B5F'><b>Block 78 dataset </b> </font>

In [49]:
# Reading block_78 data
df_block_78 = pd.read_csv("FinalData_SmartMeter/block_78.csv", engine='python')

In [50]:
# Renaming the columns
df_block_78.columns = ['LCLid', 'DateTime', 'KWh']

In [51]:
# Creating date, time related columns
df_block_78['DateTime'] = pd.to_datetime(df_block_78['DateTime'])
df_block_78['year'] = df_block_78['DateTime'].dt.year
df_block_78['month'] = df_block_78['DateTime'].dt.month
df_block_78['day'] = df_block_78['DateTime'].dt.day
df_block_78['hour'] = df_block_78['DateTime'].dt.hour
df_block_78['date'] = df_block_78['DateTime'].dt.date

In [52]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_78['time'] = df_block_78['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 78 with Household_info </b> </font>

In [53]:
df_block_78_info = df_block_78.merge(df_household, on='LCLid', how='left')

In [54]:
df_block_78_info.drop(['file'], axis=1, inplace=True)

In [55]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_78_info['Acorn'] = df_block_78_info['Acorn'].str.replace('ACORN-', "")

In [56]:
df_block_78_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000101,2011-12-12 19:00:00,0.219,2011,12,12,19,2011-12-12,07:00 PM,Std,L,Adversity
1,MAC000101,2011-12-12 19:30:00,0.202,2011,12,12,19,2011-12-12,07:30 PM,Std,L,Adversity


In [57]:
df_block_78_info['Acorn_grouped'].unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Block 79 dataset</b> </font>

In [58]:
# Reading block_79 data
df_block_79 = pd.read_csv("FinalData_SmartMeter/block_79.csv", engine='python')

In [59]:
# Renaming the columns
df_block_79.columns = ['LCLid', 'DateTime', 'KWh']

In [60]:
# Creating date, time related columns
df_block_79['DateTime'] = pd.to_datetime(df_block_79['DateTime'])
df_block_79['year'] = df_block_79['DateTime'].dt.year
df_block_79['month'] = df_block_79['DateTime'].dt.month
df_block_79['day'] = df_block_79['DateTime'].dt.day
df_block_79['hour'] = df_block_79['DateTime'].dt.hour
df_block_79['date'] = df_block_79['DateTime'].dt.date

In [61]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_79['time'] = df_block_79['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 79 with Household_info </b> </font>

In [62]:
df_block_79_info = df_block_79.merge(df_household,on='LCLid',how='left')

In [63]:
df_block_79_info.drop(['file'], axis=1, inplace=True)

In [64]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_79_info['Acorn'] = df_block_79_info['Acorn'].str.replace('ACORN-', "")

In [65]:
df_block_79_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000014,2012-10-15 11:30:00,0.178,2012,10,15,11,2012-10-15,11:30 AM,ToU,L,Adversity
1,MAC000014,2012-10-15 12:00:00,0.064,2012,10,15,12,2012-10-15,12:00 PM,ToU,L,Adversity


In [66]:
df_block_79_info.Acorn_grouped.unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Block 80 dataset</b> </font>

In [67]:
# Reading block_80 data
df_block_80 = pd.read_csv("FinalData_SmartMeter/block_80.csv", engine='python')

In [68]:
# Renaming columns
df_block_80.columns = ['LCLid', 'DateTime', 'KWh']

In [69]:
# Creating date, time related columns
df_block_80['DateTime'] = pd.to_datetime(df_block_80['DateTime'])
df_block_80['year'] = df_block_80['DateTime'].dt.year
df_block_80['month'] = df_block_80['DateTime'].dt.month
df_block_80['day'] = df_block_80['DateTime'].dt.day
df_block_80['hour'] = df_block_80['DateTime'].dt.hour
df_block_80['date'] = df_block_80['DateTime'].dt.date

In [70]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_80['time'] = df_block_80['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 80 with Household_info</b> </font>

In [71]:
df_block_80_info = df_block_80.merge(df_household, on='LCLid', how='left')

In [72]:
df_block_80_info.drop(['file'], axis=1, inplace=True)

In [73]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_80_info['Acorn'] = df_block_80_info['Acorn'].str.replace('ACORN-', "")

In [74]:
df_block_80_info.head(2)


,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000038,2011-12-08 09:00:00,0.075,2011,12,8,9,2011-12-08,09:00 AM,Std,L,Adversity
1,MAC000038,2011-12-08 09:30:00,0.081,2011,12,8,9,2011-12-08,09:30 AM,Std,L,Adversity


In [75]:
df_block_80_info['Acorn_grouped'].unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Block 95 dataset</b> </font>

In [76]:
# Reading block_95 data
df_block_95 = pd.read_csv("FinalData_SmartMeter/block_95.csv", engine='python')

In [77]:
# Renaming the columns
df_block_95.columns = ['LCLid', 'DateTime', 'KWh']

In [78]:
# Creating date, time related columns
df_block_95['DateTime'] = pd.to_datetime(df_block_95['DateTime'])
df_block_95['year'] = df_block_95['DateTime'].dt.year
df_block_95['month'] = df_block_95['DateTime'].dt.month
df_block_95['day'] = df_block_95['DateTime'].dt.day
df_block_95['hour'] = df_block_95['DateTime'].dt.hour
df_block_95['date'] = df_block_95['DateTime'].dt.date

In [79]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_95['time'] = df_block_95['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 95 with Household_info </b> </font>

In [80]:
df_block_95_info = df_block_95.merge(df_household, on='LCLid', how='left')

In [81]:
df_block_95_info.drop(['file'], axis=1, inplace=True)

In [82]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_95_info['Acorn'] = df_block_95_info['Acorn'].str.replace('ACORN-', "")

In [83]:
df_block_95_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000006,2012-01-30 11:30:00,0.096,2012,1,30,11,2012-01-30,11:30 AM,Std,Q,Adversity
1,MAC000006,2012-01-30 12:00:00,0.101,2012,1,30,12,2012-01-30,12:00 PM,Std,Q,Adversity


In [84]:
df_block_95_info['Acorn_grouped'].unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Block 96 dataset</b> </font>

In [85]:
# Reading block_96 data
df_block_96 = pd.read_csv("FinalData_SmartMeter/block_96.csv", engine='python')

In [86]:
# Renaming the columns
df_block_96.columns = ['LCLid', 'DateTime', 'KWh']

In [87]:
# Creating date, time related columns
df_block_96['DateTime'] = pd.to_datetime(df_block_96['DateTime'])
df_block_96['year'] = df_block_96['DateTime'].dt.year
df_block_96['month'] = df_block_96['DateTime'].dt.month
df_block_96['day'] = df_block_96['DateTime'].dt.day
df_block_96['hour'] = df_block_96['DateTime'].dt.hour
df_block_96['date'] = df_block_96['DateTime'].dt.date

In [88]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_96['time'] = df_block_96['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 96 with Household_info </b> </font>

In [89]:
df_block_96_info = df_block_96.merge(df_household, on='LCLid', how='left')

In [90]:
df_block_96_info.drop(['file'], axis=1, inplace=True)

In [91]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_96_info['Acorn'] = df_block_96_info['Acorn'].str.replace('ACORN-', "")

In [92]:
df_block_96_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000054,2011-12-09 09:00:00,0.161,2011,12,9,9,2011-12-09,09:00 AM,Std,Q,Adversity
1,MAC000054,2011-12-09 09:30:00,0.104,2011,12,9,9,2011-12-09,09:30 AM,Std,Q,Adversity


In [93]:
df_block_96_info['Acorn_grouped'].unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Block 105 dataset </b> </font>

In [94]:
# Reading block_105 data
df_block_105 = pd.read_csv("FinalData_SmartMeter/block_105.csv", engine='python')

In [95]:
# Renaming the columns
df_block_105.columns = ['LCLid', 'DateTime', 'KWh']

In [96]:
# Creating date, time related columns
df_block_105['DateTime'] = pd.to_datetime(df_block_105['DateTime'])
df_block_105['year'] = df_block_105['DateTime'].dt.year
df_block_105['month'] = df_block_105['DateTime'].dt.month
df_block_105['day'] = df_block_105['DateTime'].dt.day
df_block_105['hour'] = df_block_105['DateTime'].dt.hour
df_block_105['date'] = df_block_105['DateTime'].dt.date

In [97]:
# Creating time column which has time in 12 hour with AM/PM format
df_block_105['time'] = df_block_105['DateTime'].dt.time.apply(lambda x: x.strftime(format))

<font color='#114B5F'><b>Merge Block 105 with Household_info </b> </font>

In [98]:
df_block_105_info = df_block_105.merge(df_household, on='LCLid', how='left')

In [99]:
df_block_105_info.drop(['file'], axis=1, inplace=True)

In [100]:
# Cleaning: Converting 'ACORN-*' to '*'
df_block_105_info['Acorn'] = df_block_105_info['Acorn'].str.replace('ACORN-', "")

In [101]:
df_block_105_info.head(2)

,LCLid,DateTime,KWh,year,month,day,hour,date,time,stdorToU,Acorn,Acorn_grouped
0,MAC000092,2011-12-12 10:30:00,0.502,2011,12,12,10,2011-12-12,10:30 AM,Std,Q,Adversity
1,MAC000092,2011-12-12 11:00:00,0.098,2011,12,12,11,2011-12-12,11:00 AM,Std,Q,Adversity


In [102]:
df_block_105_info['Acorn_grouped'].unique()

array(['Adversity'], dtype=object)

<font color='#114B5F'><b>Final dataset after merging Block 0/2/4/62/78/79/80/95/96/105</b> </font> 

In [103]:
df_2 = df_block_2_info[df_block_2_info['stdorToU']=='ToU']

In [104]:
df_4 = df_block_4_info[df_block_4_info['stdorToU']=='ToU']

In [105]:
df_62 = df_block_62_info[df_block_62_info['stdorToU']=='ToU']

In [106]:
df_78 = df_block_78_info[df_block_78_info['stdorToU']=='ToU']

In [107]:
df_79 = df_block_79_info[df_block_79_info['stdorToU']=='ToU']

In [108]:
df_80 = df_block_80_info[df_block_80_info['stdorToU']=='ToU']

In [109]:
df_95 = df_block_95_info[df_block_95_info['stdorToU']=='ToU']

In [110]:
df_96 = df_block_96_info[df_block_96_info['stdorToU']=='ToU']

In [111]:
df_105 = df_block_105_info[df_block_105_info['stdorToU']=='ToU']

In [112]:
df_final_data = pd.concat([df_2, df_4, df_62, df_78, df_79, df_80, df_95, df_96, df_105, df_block_0_info])

In [113]:
# Renaming a column
df_final_data = df_final_data.rename(columns={'stdorToU' : 'Std or ToU tariff'})

In [114]:
# Number of households using Dynamic Time of Use tariff plan
df_final_data[df_final_data['Std or ToU tariff']=='ToU']['LCLid'].nunique()

174

In [115]:
# Number of households using Standard tariff plan
df_final_data[df_final_data['Std or ToU tariff']=='Std']['LCLid'].nunique()

388

In [116]:
# Creating a column with quarter information
df_final_data['quarter'] = df_final_data['DateTime'].dt.quarter

In [117]:
df_final_data['quarter'] = df_final_data['year'].astype(str) +' Q'+ df_final_data['quarter'].astype(str)

In [118]:
#Converting string to numeric type
df_final_data['KWh'] = df_final_data['KWh'].convert_objects(convert_numeric=True)

In [119]:
# Creating a CSV file.
df_final_data.to_csv("final.csv", sep=',', index=False, encoding='utf-8')